## 기말 과제 

- 이메일 제출: jihoyeo@gachon.ac.kr
- Python 코드 및 React 개발 코드를 압축하여 이름_학번.zip 형태로 제출
- https://stackblitz.com 로 React를 빌드하여 시뮬레이션을 구축하고, 해당 URL을 첨부할 것 (e.g., https://testsimulation4-attq--3000--d3acb9e1.local-credentialless.webcontainer.io)

## [1] 주제 및 데이터

### 1.1. 데이터
- 아래 데이터는 가천대학교 학생들이 하루동안 이동한 예시 데이터입니다. 이 데이터는 다음과 같은 항목을 포함하고 있습니다. 
    - `stops_point.json` : 무당이가 정차하는 Station들의 위치
    - `station_schedule.json` : Station 마다 무당이가 출발하는 시간
    - `OD_data_ladybug.shp` : 가천대학교 학생 300명의 가상의 이동데이터
    - `distance_by_stops_geo`: 무당이 정류장 사이의 거리 및 이동시간 (무당이 탑승 기준) 
- 출발위치와 도착위치는 가천대학교에서 운영하는 무당이의 정류소 기준이며, 각 정류소의 좌표는 `station_info.csv`에서 확인할 수 있습니다.

In [2]:
import json
import os
import pandas as pd
import geopandas as gpd

In [9]:
path = './data_final/'

# Load the JSON file
with open(os.path.join(path + 'stops_point.json'), "r", encoding="utf-8") as json_file:
    stops_point = json.load(json_file)

with open(os.path.join(path + 'station_schedule.json'), "r", encoding="utf-8") as json_file:
    station_schedule = json.load(json_file)

- `stops_point` 데이터는 무당이가 정차하는 station의 위경도 좌표를 포함합니다

In [10]:
stops_point

{'가천대_ai공학관': [127.133374, 37.455009],
 '가천대_총학생회': [127.133923, 37.453348],
 '가천대_일반대학원': [127.130112, 37.452589],
 '가천대_반도체대학': [127.127384, 37.45091],
 '가천대_학생회관': [127.130121, 37.450268],
 '가천대_교육대학원': [127.131698, 37.452066]}

- `station_schedule`은 무당이가 해당 역에서 출발하는 시간을 나타냅니다. 가천대 AI 공학관에서 10분간격으로 정시에 출발한다고 가정하였습니다.
- 여기서 단위는 '분' 입니다. 예를들어 값이 600이란 이야기는 오전 10시에 AI공학관을 출발한다는 이야기입니다. 다음 버스는 610분(10시 10분)에 출발하겠네요.

In [17]:
station_schedule['가천대_ai공학관']

[600.0,
 610.0,
 620.0,
 630.0,
 640.0,
 650.0,
 660.0,
 670.0,
 680.0,
 690.0,
 700.0,
 710.0,
 720.0,
 730.0,
 740.0,
 750.0,
 760.0,
 770.0,
 780.0,
 790.0,
 800.0,
 810.0,
 820.0,
 830.0,
 840.0,
 850.0,
 860.0,
 870.0,
 880.0,
 890.0,
 900.0,
 910.0,
 920.0,
 930.0,
 940.0,
 950.0,
 960.0,
 970.0,
 980.0,
 990.0,
 1000.0,
 1010.0,
 1020.0,
 1030.0,
 1040.0,
 1050.0,
 1060.0,
 1070.0,
 1080.0]

- `OD_data_ladybug.shp`은 가천대학교 학생들 300명의 가상의 이동데이터 입니다.
- 출발시간과, 탑승위치(Origin), 하차위치(Destination) 정보를 포함하고 있습니다.
- `ladybug_path`와 `geometry`컬럼은 이들이 만약 무당이를 탄다고 했을 때 거쳐가는 정류장과 그 Linestring을 나타냅니다.
- 하지만, 이들은 아직 무당이를 탈지, 그냥 걸어서 갈지 결정하지 못한 상황입니다.
    - 도보를 선택한다면 출발시간에 바로 목적지로 출발하면 됩니다.
    - 무당이를 이용하기로 결정한다면 해당 탑승위치에서, 무당이가 올 때까지 기다려야 합니다. 무당이가 출발하는 시간은 `station_schedule`에서 확인할 수 있습니다.

In [3]:
from shapely.geometry import Point

path = './data_final/OD_data_ladybug'

# GeoDataFrame으로 Shapefile 읽기
OD_data_ladybug = gpd.read_file(path, encoding = 'euckr')

# 컬러명 수정
OD_data_ladybug.rename(columns={
    'start_time': 'start_time',
    'start_poin': 'start_point',
    'start_stat': 'start_station',
    'end_point': 'end_point',
    'end_statio': 'end_station',
    'ladybug_pa': 'ladybug_path'
}, inplace=True)

# 문자열을 Point 객체로 변환하는 함수
def convert_to_point(point_str):
    # 'POINT (127.127384 37.45091)' -> [127.127384, 37.45091]
    coords = point_str.replace("POINT (", "").replace(")", "").split()
    return Point(float(coords[0]), float(coords[1]))

# 'start_point'와 'end_point'를 Point 객체로 변환
OD_data_ladybug['start_point'] = OD_data_ladybug['start_point'].apply(convert_to_point)
OD_data_ladybug['end_point'] = OD_data_ladybug['end_point'].apply(convert_to_point)

OD_data_ladybug

,start_time,start_point,start_station,end_point,end_station,ladybug_path,geometry
0,661.050000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130121 37.450268),가천대_학생회관,가천대_반도체대학 -> 가천대_학생회관,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
1,679.083333,POINT (127.133923 37.453348),가천대_총학생회,POINT (127.127384 37.45091),가천대_반도체대학,가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대학,"LINESTRING (127.13390 37.45337, 127.13375 37.4..."
2,641.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130112 37.452589),가천대_일반대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학...,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
3,632.150000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.133374 37.455009),가천대_ai공학관,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
4,618.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.131698 37.452066),가천대_교육대학원,가천대_반도체대학 -> 가천대_학생회관 -> 가천대_교육대학원,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
...,...,...,...,...,...,...,...
295,999.450000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.130112 37.452589),가천대_일반대학원,가천대_교육대학원 -> 가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원,"LINESTRING (127.13181 37.45222, 127.13176 37.4..."
296,988.933333,POINT (127.130121 37.450268),가천대_학생회관,POINT (127.133374 37.455009),가천대_ai공학관,가천대_학생회관 -> 가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13001 37.45030, 127.13007 37.4..."
297,1002.350000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.133374 37.455009),가천대_ai공학관,가천대_교육대학원 -> 가천대_ai공학관,"LINESTRING (127.13181 37.45222, 127.13176 37.4..."
298,1024.083333,POINT (127.133374 37.455009),가천대_ai공학관,POINT (127.131698 37.452066),가천대_교육대학원,가천대_ai공학관 -> 가천대_총학생회 -> 가천대_일반대학원 -> 가천대_반도체대...,"LINESTRING (127.13322 37.45500, 127.13325 37.4..."


In [18]:
OD_data_ladybug

,출발시간,탑승위치(Station_id),탑승위치(정류장 이름),하차위치(Station_id),하차위치(정류장 이름)
0,661.050000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130121 37.450268),가천대_학생회관
1,679.083333,POINT (127.133923 37.453348),가천대_총학생회,POINT (127.127384 37.45091),가천대_반도체대학
2,641.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.130112 37.452589),가천대_일반대학원
3,632.150000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.133374 37.455009),가천대_ai공학관
4,618.350000,POINT (127.127384 37.45091),가천대_반도체대학,POINT (127.131698 37.452066),가천대_교육대학원
...,...,...,...,...,...
295,999.450000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.130112 37.452589),가천대_일반대학원
296,988.933333,POINT (127.130121 37.450268),가천대_학생회관,POINT (127.133374 37.455009),가천대_ai공학관
297,1002.350000,POINT (127.131698 37.452066),가천대_교육대학원,POINT (127.133374 37.455009),가천대_ai공학관
298,1024.083333,POINT (127.133374 37.455009),가천대_ai공학관,POINT (127.131698 37.452066),가천대_교육대학원


`distance_by_stops_geo`은 무당이의 정류장간 이동 경로와 걸리는 시간을 나타낸 데이터입니다. 속도는 17로 가정하였습니다.
- 여기서 time의 단위는 '분' 이고 distance_m의 단위는 'm'입니다.

In [4]:
path = './data_final/distance_by_stops_geo'

# GeoDataFrame으로 Shapefile 읽기
distance_by_stops_geo = gpd.read_file(path, encoding = 'euckr')

distance_by_stops_geo

,start,end,distance_m,time,geometry
0,가천대_ai공학관,가천대_총학생회,204.3,0.721059,"LINESTRING (127.13322 37.45500, 127.13325 37.4..."
1,가천대_총학생회,가천대_일반대학원,424.5,1.498235,"LINESTRING (127.13390 37.45337, 127.13375 37.4..."
2,가천대_일반대학원,가천대_반도체대학,352.9,1.245529,"LINESTRING (127.13017 37.45249, 127.13004 37.4..."
3,가천대_반도체대학,가천대_학생회관,298.0,1.051765,"LINESTRING (127.12748 37.45091, 127.12749 37.4..."
4,가천대_학생회관,가천대_교육대학원,299.6,1.057412,"LINESTRING (127.13001 37.45030, 127.13007 37.4..."
5,가천대_교육대학원,가천대_ai공학관,445.3,1.571647,"LINESTRING (127.13181 37.45222, 127.13176 37.4..."


### 1.2 분석 주제
여러분은 이제 가천대학교 학생들의 Trip Planner가 되어야 합니다. 아래에 대한 솔루션을 제시해주세요. 
1. 개별 통행마다, 무당이를 타는 것이 시간이 절약될지, 걸어가는 것이 시간이 절약될지 판단해주세요. (10점)
    - OD_data_ladybug에 `교통수단`이라는 컬럼을 만들어서 '무당이', '도보'를 구분해주세요.
    - 앞서 언급하였듯이 무당이의 속도는 17km/h 입니다. 도보는 5km/h로 가정합시다. 도보의 Route는 제공되지 않았습니다. OSRM API를 사용해 도보의 경로 및 시간을 뽑아야 합니다. 
2. 학생들은 통행시간이 적게 걸리는 통행을 선택할겁니다. 무당이를 타는 학생과, 걸어가는 학생들을 Simulation 프레임워크를 이용해서 시각화 해보세요. (10점)
    - 시뮬레이션은 Stackbliz를 통해 시각화 해야 합니다. 
    - 구축한 시뮬레이션의 웹호스팅 링크와 코드파일을 함께 제출해주세요. 
3. 총 몇명의 학생들이 무당이를 이용하고, 몇명의 학생들이 도보를 선택했나요? (10점)
4. 무당이를 이용한 학생들의 평균 대기시간은 몇분인가요? (10점)

## [2] 분석결과

- 위의 4가지 문제에 대해서, 이곳에 여러분의 코드를 붙이고 시뮬레이션을 구현해보도록 합시다.